## 📚 Prerequisites

Before executing this notebook, make sure you have properly set up your Azure Services, created your Conda environment, and configured your environment variables as per the instructions provided in the [README.md](README.md) file.

## 📋 Table of Contents

Explore different retrieval methods in Azure AI Search:

1. [**Understanding Types of Search**](#define-field-types): This section provides a comprehensive overview of the different types of search methods available in Azure AI Search.
2. [**Keyword Search**](#keyword-search): Use direct query term matching with document content.
3. [**Vector Search**](#vector-search): Employ embeddings for semantic content understanding and relevance ranking.
4. [**Hybrid Search**](#hybrid-search): Combine keyword and vector search for comprehensive results.
5. [**Reranking Search**](#reranking-search): Reorder initial search results for improved top result relevance.

Additional resources:
- [Azure AI Search Documentation](https://learn.microsoft.com/en-us/azure/search/)

### 🧭 Understanding Types of Search  

+ **Keyword Search**: Traditional search method relying on direct term matching. Efficient for exact matches but struggles with synonyms and context. [Learn More](https://learn.microsoft.com/en-us/azure/search/search-lucene-query-architecture)

- **Vector Search**: Converts text into high-dimensional vectors to understand semantic meaning. Finds relevant documents even without exact keyword matches. Effectiveness depends on quality of training data. [Learn More](https://learn.microsoft.com/en-us/azure/search/vector-search-overview)

+ **Hybrid Search**: Combines Keyword and Vector Search for comprehensive, contextually relevant results. Effective for complex queries requiring nuanced understanding. [Learn More](https://learn.microsoft.com/en-us/azure/search/vector-search-ranking#hybrid-search)

- **Reranking Search**: Fine-tunes initial search results using advanced algorithms for relevance. Useful when initial retrieval returns relevant but not optimally ordered results. [Learn More](https://learn.microsoft.com/en-us/azure/search/semantic-search-overview)

### 🚧 Limitations

##### Keyword Search
- **Synonym Challenges**: Struggles with recognizing synonyms or different expressions of the same concept.
- **Context Understanding**: May not fully capture the broader context or the query's intent, especially in complex queries.
##### Embedding-Based Search
- **Keyword Precision**: May miss documents that contain exact terms if those terms don't semantically align with the query or document's overall content.
- **Contextual Misinterpretations**: May overgeneralize or incorrectly interpret context, missing specific nuances.
- **Training Data Dependency**: Performance heavily relies on the diversity and depth of the training data.
### 💡 Recommendations

To achieve higher relevance out of the box: 

1. **Hybrid Search**: Combines keyword and vector search methods to ensure comprehensive document retrieval across a range of queries, from highly specific to semantically complex.

2. **Re-Ranking and L2 in AI Search**: Enhances initial search results by applying sophisticated ranking algorithms, improving relevance and accuracy, especially for nuanced queries.

In [1]:
import os
from dotenv import load_dotenv
import os
from dotenv import load_dotenv
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.models import (
    VectorQuery,
    VectorizedQuery,
    VectorizableTextQuery,
    QueryType,
    QueryCaptionType,
    QueryAnswerType,
)

# Load environment variables from .env file
load_dotenv()

# Define the target directory
target_directory = os.getcwd()  # Get the current working directory

# Move one directory back
parent_directory = os.path.dirname(target_directory)

# Check if the parent directory exists
if os.path.exists(parent_directory):
    # Change the current working directory to the parent directory
    os.chdir(parent_directory)
    print(f"Directory changed to {os.getcwd()}")
else:
    print(f"Parent directory {parent_directory} does not exist.")

Directory changed to c:\Users\pablosal\Desktop\aihlsignited-medindexer


In [2]:
# Set up Azure Cognitive Search credentials
service_endpoint = os.getenv("AZURE_AI_SEARCH_SERVICE_ENDPOINT")
key = os.getenv("AZURE_SEARCH_ADMIN_KEY")
credential = AzureKeyCredential(key)

# Define the name of the Azure Search index
# This is the index where your data is stored in Azure Search
index_name = os.getenv("AZURE_SEARCH_INDEX_NAME")

# Set up the Azure Search client with the specified index
# This prepares the client to interact with the Azure Search service
search_client = SearchClient(service_endpoint, index_name, credential=credential)

# Set the service endpoint and API key from the environment
# Create an SDK client
from src.aoai.aoai_helper import AzureOpenAIManager
aoai_client = AzureOpenAIManager()

search_query = "What is the prior authorization policy for Inflammatory Conditions?"
search_vector = aoai_client.generate_embedding(search_query)

2025-04-02 22:14:50,681 - micro - MainProcess - ERROR    API Connection Error: The server could not be reached. (aoai_helper.py:generate_embedding:793)
2025-04-02 22:14:50,683 - micro - MainProcess - ERROR    Error details: Connection error. (aoai_helper.py:generate_embedding:794)
2025-04-02 22:14:50,974 - micro - MainProcess - ERROR    Traceback: Traceback (most recent call last):
  File "c:\Users\pablosal\AppData\Local\anaconda3\envs\mediindexer\lib\site-packages\httpx\_transports\default.py", line 72, in map_httpcore_exceptions
    yield
  File "c:\Users\pablosal\AppData\Local\anaconda3\envs\mediindexer\lib\site-packages\httpx\_transports\default.py", line 236, in handle_request
    resp = self._pool.handle_request(req)
  File "c:\Users\pablosal\AppData\Local\anaconda3\envs\mediindexer\lib\site-packages\httpcore\_sync\connection_pool.py", line 256, in handle_request
    raise exc from None
  File "c:\Users\pablosal\AppData\Local\anaconda3\envs\mediindexer\lib\site-packages\httpcore\

## Keyword Search 

**Full-text search**: This method uses the `@search.score` parameter and the BM25 algorithm for scoring. The BM25 algorithm is a bag-of-words retrieval function that ranks a set of documents based on the query terms appearing in each document, regardless of their proximity within the document. There is no upper limit for the score in this method.

```json
"value": [
 {
    "@search.score": 5.1958685,
    "@search.features": {
        "description": {
            "uniqueTokenMatches": 1.0,
            "similarityScore": 0.29541412,
            "termFrequency" : 2
        },
        "title": {
            "uniqueTokenMatches": 3.0,
            "similarityScore": 1.75451557,
            "termFrequency" : 6
        }
    }
 }
]
 ```

- `uniqueTokenMatches`: This parameter indicates the number of unique query terms found in the document field. A higher value means more unique query terms were found, suggesting a stronger match.

- `similarityScore`: This parameter represents the semantic similarity between the content of the document field and the query terms. A higher `similarityScore` means the document content is more semantically similar to the query, indicating a more relevant match.

- `termFrequency`: This parameter shows how often the query terms appear within the document field. A higher `termFrequency` means the query terms appear more often, suggesting a stronger match.

These parameters contribute to the overall `@search.score`. The `@search.score` is a cumulative measure of a document's relevance to the search query. A higher `@search.score` indicates a stronger match between the document and the search query.

When interpreting search results, documents with higher scores are generally considered more relevant to the query than those with lower scores.

In [10]:
# keyword search
r = search_client.search(search_query, top=5)
for doc in r:
    if "Inflammatory Conditions" in doc["chunk"]:
        content = doc["chunk"].replace("\n", " ")[:1000]
        print(f"score: {doc['@search.score']}. {content}")

score: 5.676634. 1\. Ankylosing Spondylitis. Approve for the duration noted if the patient meets ONE of the following (A or B):   A) Initial Therapy. Approve for 6 months if the patient meets BOTH of the following (i and ii):   i. Patient is ≥ 18 years of age; AND   ii. The medication is prescribed by or in consultation with a rheumatologist.   B) Patient is Currently Receiving an Adalimumab Product. Approve for 1 year if the patient meets BOTH of the following (i and ii):   i. Patient has been established on therapy for at least 6 months; AND Note: A patient who has received < 6 months of therapy or who is restarting therapy with an adalimumab product is reviewed under criterion A (Initial Therapy).   ii. Patient meets at least ONE of the following (a or b):   a) When assessed by at least one objective measure, patient experienced a beneficial clinical response from baseline (prior to initiating an adalimumab product); OR Note: Examples of objective measures include Ankylosing Spondyl

## Vector Search 

This method also uses the `@search.score` parameter but uses the HNSW (Hierarchical Navigable Small World) algorithm for scoring. The HNSW algorithm is an efficient method for nearest neighbor search in high dimensional spaces. The scoring range is 0.333 - 1.00 for Cosine similarity, and 0 to 1 for Euclidean and DotProduct similarities.

In [13]:
# vector search
r = search_client.search(
    top=5,
    vector_queries=[
        VectorizedQuery(vector=search_vector.data[0].embedding, k_nearest_neighbors=50, fields="vector", weight=0.5),
    ],
)

# Iterate through the search results and print all metadata
for doc in r:
    content = doc["chunk"].replace("\n", " ")[:1000]
    print(
        f"score: {doc['@search.score']}, reranker: {doc['@search.reranker_score']}. {content}"
    )

score: 0.77632785, reranker: None. POLICY:   Inflammatory Conditions - Adalimumab Products Prior Authorization   Policy   · Abrilada™ (adalimumab-afzb subcutaneous injection - Pfizer)   · adalimumab-aacf subcutaneous injection (Fresenius Kabi)   · adalimumab-adaz subcutaneous injection (Sandoz/Novartis)   · adalimumab-adbm subcutaneous injection (Boehringer Ingelheim)   · adalimumab-fkjp subcutaneous injection (Mylan)   · adalimumab-ryvk subcutaneous injection (Teva/Alvotech)   · Amjevita® (adalimumab-atto subcutaneous injection - Amgen)   · Cyltezo® (adalimumab-adbm subcutaneous injection - Boehringer Ingelheim)   · Hadlima™ (adalimumab-bwwd subcutaneous injection –   Organon/Samsung Bioepis)   · Hulio® (adalimumab-fkjp subcutaneous injection - Mylan)   · Humira® (adalimumab subcutaneous injection - AbbVie, Cordavis)   . Hyrimoz® (adalimumab-adaz subcutaneous Sandoz/Novartis, Cordavis) injection –   · Idacio® (adalimumab-aacf subcutaneous injection - Fresenius Kabi)   · Simlandi® (ada

## Hybrid search

This method uses the `@search.score` parameter and the RRF (Reciprocal Rank Fusion) algorithm for scoring. The RRF algorithm is a method for data fusion that combines the results of multiple queries. The upper limit of the score is bounded by the number of queries being fused, with each query contributing a maximum of approximately 1 to the RRF score. For example, merging three queries would produce higher RRF scores than if only two search results are merged.

In [14]:
# Hybrid retrieval + rerank
r = search_client.search(
    search_text=search_query,
    top=5,
    vector_queries=[
        VectorizedQuery(vector=search_vector.data[0].embedding, k_nearest_neighbors=50, fields="vector", weight=0.5),
    ],
)

# Iterate through the search results and print all metadata
for doc in r:
    content = doc["chunk"].replace("\n", " ")[:1000]
    print(
        f"score: {doc['@search.score']}, reranker: {doc['@search.reranker_score']}. {content}"
    )

score: 0.024242425337433815, reranker: None. 1\. Ankylosing Spondylitis. Approve for the duration noted if the patient meets ONE of the following (A or B):   A) Initial Therapy. Approve for 6 months if the patient meets BOTH of the following (i and ii):   i. Patient is ≥ 18 years of age; AND   ii. The medication is prescribed by or in consultation with a rheumatologist.   B) Patient is Currently Receiving an Adalimumab Product. Approve for 1 year if the patient meets BOTH of the following (i and ii):   i. Patient has been established on therapy for at least 6 months; AND Note: A patient who has received < 6 months of therapy or who is restarting therapy with an adalimumab product is reviewed under criterion A (Initial Therapy).   ii. Patient meets at least ONE of the following (a or b):   a) When assessed by at least one objective measure, patient experienced a beneficial clinical response from baseline (prior to initiating an adalimumab product); OR Note: Examples of objective measure

#### Enable Exhaustive `ExhaustiveKnn`

In [ ]:
# Vector retrieval
r = search_client.search(
    search_text=search_query,
    top=5,
    vector_queries=[
        VectorizedQuery(vector=search_vector.data[0].embedding, exhaustive=True, k_nearest_neighbors=50, fields="vector", weight=0.5),
    ],
)

# Iterate through the search results and print all metadata
for doc in r:
    content = doc["chunk"].replace("\n", " ")[:1000]
    print(
        f"score: {doc['@search.score']}, reranker: {doc['@search.reranker_score']}. {content}"
    )

score: 0.024242425337433815, reranker: None. 1\. Ankylosing Spondylitis. Approve for the duration noted if the patient meets ONE of the following (A or B):   A) Initial Therapy. Approve for 6 months if the patient meets BOTH of the following (i and ii):   i. Patient is ≥ 18 years of age; AND   ii. The medication is prescribed by or in consultation with a rheumatologist.   B) Patient is Currently Receiving an Adalimumab Product. Approve for 1 year if the patient meets BOTH of the following (i and ii):   i. Patient has been established on therapy for at least 6 months; AND Note: A patient who has received < 6 months of therapy or who is restarting therapy with an adalimumab product is reviewed under criterion A (Initial Therapy).   ii. Patient meets at least ONE of the following (a or b):   a) When assessed by at least one objective measure, patient experienced a beneficial clinical response from baseline (prior to initiating an adalimumab product); OR Note: Examples of objective measure

## Semantic ranking

This method uses the `@search.rerankerScore` parameter and a semantic ranking algorithm for scoring. Semantic ranking is a method that uses machine learning models to understand the semantic content of the queries and documents, and ranks the documents based on their relevance to the query. The scoring range is 0.00 - 4.00 in this method.

Remember, a higher score indicates a higher relevance of the document to the query.

In [18]:
# Hybrid retrieval + rerank
r = search_client.search(
    search_text=search_query,
    top=5,
    vector_queries=[
        VectorizedQuery(vector=search_vector.data[0].embedding, exhaustive=True, k_nearest_neighbors=50, fields="vector", weight=0.5),
    ],
    query_type=QueryType.SEMANTIC,
    semantic_configuration_name="policy-index-semantic-config",
    query_language="en-us",
    query_caption=QueryCaptionType.EXTRACTIVE,
    query_answer=QueryAnswerType.EXTRACTIVE,
)

# Iterate through the search results and print all metadata
for doc in r:
    content = doc["chunk"].replace("\n", " ")[:1000]
    print(
        f"score: {doc['@search.score']}, reranker: {doc['@search.reranker_score']}. {content}"
    )

ServiceRequestError: <urllib3.connection.HTTPSConnection object at 0x0000027A5B32EE30>: Failed to resolve 'search-ai-factory-centralus.search.windows.net' ([Errno 11002] getaddrinfo failed)